In [8]:
import re
import jieba
import pandas as pd

# gensim训练词向量：

1. 读取预处理好的数据
2. 训练
3. 完事

# 1. 数据集路径

In [33]:
merger_data_path = 'data/merged_train_test_seg_data.csv'

# 2. 载入数据

In [11]:
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
print('train data size {},test data size {}'.format(len(train_df),len(test_df)))
train_df.head()

train data size 82871,test data size 20000


,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机 重 助力 泵 方向机 都 换,新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问...,随时 联系
1,Q2,奔驰,奔驰M级,奔驰 ML500 排气 凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马X1 进口,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位 ...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,Jeep,牧马人,30V6 发动机 号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰C级,2012 款 奔驰 c180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场 价钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


# 3. 模型创建

Gensim中 Word2Vec 模型的期望输入是进过分词的句子列表，即是某个二维数组。这里我们暂时使用 Python 内置的数组，不过其在输入数据集较大的情况下会占用大量的 RAM。Gensim 本身只是要求能够迭代的有序句子列表，因此在工程实践中我们可以使用自定义的生成器，只在内存中保存单条语句。

## Word2Vec 参数
+ min_count

在不同大小的语料集中，我们对于基准词频的需求也是不一样的。譬如在较大的语料集中，我们希望忽略那些只出现过一两次的单词，这里我们就可以通过设置min_count参数进行控制。一般而言，合理的参数值会设置在0~100之间。

+ size

size参数主要是用来设置神经网络的层数，Word2Vec 中的默认值是设置为100层。更大的层次设置意味着更多的输入数据，不过也能提升整体的准确度，合理的设置范围为 10~数百。

+ workers

workers参数用于设置并发训练时候的线程数，不过仅当Cython安装的情况下才会起作用：

In [12]:
# 引入 word2vec
from gensim.models.word2vec import LineSentence
from gensim.models import word2vec
import gensim

# 引入日志配置
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 构建训练

In [ ]:
model = word2vec.Word2Vec(LineSentence(merger_data_path), workers=8,min_count=0,size=200)

# 查找最近的词

In [15]:
model.wv.most_similar(['奇瑞'],topn=10)

[('海马', 0.8893340826034546),
 ('东南', 0.876111626625061),
 ('二代', 0.8668296337127686),
 ('铃木', 0.8638443946838379),
 ('名爵', 0.8610075116157532),
 ('猎豹', 0.856071412563324),
 ('东风风行', 0.8480671048164368),
 ('江淮', 0.846108615398407),
 ('瑞虎5', 0.8452643752098083),
 ('东风', 0.8447585701942444)]

# 保存模型

In [34]:
save_model_path='data/wv/word2vec.model'

In [ ]:
model.save(save_model_path)

# 载入模型

In [ ]:
model = word2vec.Word2Vec.load(save_model_path)

In [ ]:
model.wv.most_similar(['奇瑞'],topn=10)

# 参考

1. https://radimrehurek.com/gensim/models/word2vec.html 